# Text Summarization Demo

## Description

This notebook provides an interactive demo for text summarization using pre-trained Llama models from Hugging Face's `transformers` library, integrated with `gradio` for a simple and user-friendly interface.

### Main Features:
- Select from a variety of pre-trained Llama models for text summarization.
- Input any text and generate a concise summary.
- Display the summary in a dedicated output textbox.
- Cache models to avoid reloading and improve performance.

## Log in to Hugging Face Hub

In this cell, we import the `login` function from the Hugging Face Hub and call it to authenticate with your Hugging Face account. This step is required to access private models, datasets, and other resources hosted on Hugging Face.

### Directions to Generate Access Token:
1. Go to the [Hugging Face website](https://huggingface.co/).
2. Log in to your Hugging Face account.
3. Navigate to your **profile icon** on the top right, and click **Settings**.
4. Under **Access Tokens** (on the left sidebar), click **New Token** to generate a new access token.
5. Copy the generated token.

### Usage:
When you run this cell, you'll be prompted to paste the access token, which grants access to your Hugging Face resources.

> **Do not share your Access Tokens with anyone**

In [ ]:
from huggingface_hub import login
login()

## Import Required Libraries

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Basic Usage

Below there are a list of available Llama models to choose from. The function, `load_model`, loads the selected Llama model along with its tokenizer. It uses Hugging Face's `AutoModelForCausalLM` and `AutoTokenizer` to load the pre-trained model and then returns a tuple containing the loaded model and tokenizer. Note that, for each version of Llama you need to seperately request access.

In [ ]:
llama_models = {
    "Llama 3 70B Instruct": "meta-llama/Meta-Llama-3-70B-Instruct",
    "Llama 3 8B Instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
    "Llama 3.1 70B Instruct": "meta-llama/Llama-3.1-70B-Instruct",
    "Llama 3.1 8B Instruct": "meta-llama/Llama-3.1-8B-Instruct",
    "Llama 3.2 3B Instruct": "meta-llama/Llama-3.2-3B-Instruct",
    "Llama 3.2 1B Instruct": "meta-llama/Llama-3.2-1B-Instruct",
}

In [ ]:
def load_model(model_name):
    """Load the specified Llama model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')
    model.to(device)
    return model, tokenizer

Cache models to avoid reloading

In [ ]:
model_cache = {}

The `summarize_text` function generates a summary of the input text using the selected Llama model. It checks if the model is cached, loads it if necessary, and then formats the input text as a prompt for summarization. The model processes the input, and the function returns the generated summary by extracting it from the model's output.

In [ ]:
def summarize_text(text, model_choice):
    """Summarize text using the selected Llama model."""
    if model_choice not in model_cache:
        model_cache[model_choice] = load_model(llama_models[model_choice])
    model, tokenizer = model_cache[model_choice]
    prompt = f"Summarize the following text:\n\n\"{text}\"\n\nSummary:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=512, do_sample=False)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = generated_text.split("Summary:")[-1].strip()
    return summary

The cell below defines a simple Gradio interface for text summarization. The interface consists of a dropdown menu where users can select a specific Llama model for summarization, a textbox where users can input the text they want to summarize, and an output box to display the generated summary. Once the "Summarize" button is clicked, the selected model processes the input and returns a concise summary of the provided text.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Summarization with Llama Models</center></h1>")
    model_choice = gr.Dropdown(list(llama_models.keys()), label="Select Llama Model")
    input_text = gr.Textbox(label="Enter text to summarize", lines=10)
    output_text = gr.Textbox(label="Summary")
    summarize_button = gr.Button("Summarize")
    summarize_button.click(summarize_text, [input_text, model_choice], output_text)

Launch the interface

In [ ]:
demo.launch()